<a href="https://colab.research.google.com/github/joyo11/SeasonalInference/blob/main/DLCV_final_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchgeo crcmod

In [ ]:
import os
import gc
import random
import torch
import timm
import torchgeo
import rasterio
# import kornia
import pickle
# import tempfile
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchgeo.datasets import RasterDataset#, stack_samples, unbind_samples, Landsat8
from torchgeo import models
from torchgeo.trainers import ClassificationTask
from torchgeo.models import ResNet18_Weights
from torchgeo.samplers import RandomGeoSampler
import numpy as np
import matplotlib.pyplot as plt
# from torchvision import transforms
from datetime import datetime
# from tqdm import tqdm

In [ ]:
from google.colab import files

# Upload the service account key file
uploaded = files.upload()

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/dl-for-cv-final-project-f69fa12b7fae.json'

In [ ]:
!gcloud auth activate-service-account --key-file='/content/dl-for-cv-final-project-f69fa12b7fae.json'

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
bucket_name = 'dlcv_finalproj_data1'
folder_path = 'imgcollect'

!gsutil -m -o 'GSUtil:sliced_object_download_threshold=150M' cp -r gs://{bucket_name}/{folder_path} /content

In [ ]:
bucket_name = 'dlcv_finalproj_data1'
folder_path = 'random_samples/LC08_014029_20191211.pkl'
!gsutil -m -o 'GSUtil:sliced_object_download_threshold=150M' cp -r gs://{bucket_name}/{folder_path} /content

In [ ]:
bucket_name = 'dlcv_finalproj_data1'
file_path = 'combined_data.pkl'
!gsutil -m -o 'GSUtil:sliced_object_download_threshold=150M' cp -r gs://{bucket_name}/{file_path} /content

In [ ]:
with open("/content/combined_data.pkl", 'rb') as file:
    data = pickle.load(file)

random.shuffle(data)

In [ ]:
image, label = data[4678]
plot_image(image)
if label==1:
    month='January'
elif label ==2:
    month="February"
elif label ==3:
    month="March"
elif label ==4:
    month="April"
elif label ==5:
    month="May"
elif label ==6:
    month="June"
elif label ==7:
    month="July"
elif label ==8:
    month="August"
elif label ==9:
    month="September"
elif label ==10:
    month="October"
elif label ==11:
    month="November"
else:
    month="December"
print("Label =",label,"Month =",month)


NameError: ignored

In [ ]:
train_data = data[:10000]
val_data = data[10001:11250]
test_data = data[11250:]

In [ ]:
train_loader = DataLoader(train_data, batch_size=64)
val_loader = DataLoader(val_data, batch_size=64)
test_loader = DataLoader(test_data, batch_size=64)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
class LandsatCustom(RasterDataset):
    is_image = True
    separate_files = False
    all_bands = ["B1","B2","B3","B4","B5","B6","B7","B8","B9","B10","B11"]
    rgb_bands = ["B4", "B3", "B2"]
    filename_regex = r'(?P<satellite>[A-Za-z0-9]+)_(?P<number>\d+)_(?P<date>(?P<year>\d{4})(?P<month>\d{2})(?P<day>\d{2}))\.tif'

    def __init__(self, root, transform=None, target_transform=None):
        super(LandsatCustom, self).__init__(root)
        self.transform = transform
        self.target_transform = target_transform
        xmin = self.index.bounds[0] + (self.index.bounds[1] - self.index.bounds[0])/5
        xmax = self.index.bounds[1] - (self.index.bounds[1] - self.index.bounds[0])/5
        ymin = self.index.bounds[2] + (self.index.bounds[3] - self.index.bounds[2])/5
        ymax = self.index.bounds[3] - (self.index.bounds[3] - self.index.bounds[2])/5
        newroi = torchgeo.datasets.BoundingBox(xmin, xmax, ymin, ymax, self.index.bounds[4], self.index.bounds[5])
        # sampler = RandomGeoSampler(self, roi=newroi, size=256, length=5)
        # self.dataset = []
        # for index in sampler:
        #   sample = self[index]
        #   self.dataset.append(sample)

    def __getitem__(self, index):
        raster_data = super(LandsatCustom, self).__getitem__(index)
        image = raster_data['image'][:7,:,:]
        label = datetime.utcfromtimestamp(self.index.bounds[4]).month
        return (image, label)

    def get_samples(self):
        idx = random.randint(0, 4)
        return self.dataset[idx]

In [ ]:
image = LandsatCustom('/content/drive/MyDrive/DLCVProj')

In [ ]:
sampler = RandomGeoSampler(image, roi=torchgeo.datasets.BoundingBox(xmin, xmax, ymin, ymax, self.index.bounds[4], self.index.bounds[5]), size=256, length=5)

In [ ]:
class CombinedDataset(torch.utils.data.Dataset):
    def __init__(self, instances_dict, len):
        self.instances_dict = instances_dict
        self.length = len

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        chosen_key = random.choice(list(self.instances_dict.keys()))
        chosen_instance = self.instances_dict[chosen_key]
        sample = chosen_instance.get_samples()
        return sample

In [ ]:
def plot_image(image):
        rgb_indices = [3,2,1]
        image = image[rgb_indices].permute(1, 2, 0)
        image = ((image - 6000) / 6000).clip(0, 1).numpy()
        fig, ax = plt.subplots()
        ax.imshow(image)
        ax.set_xticks([])
        ax.set_yticks([])

In [ ]:
# from pathlib import Path

# def is_file_corrupted(file_path):
#     try:
#         with open(file_path, 'rb') as file:
#             pickle.load(file)
#         return False  # File is not corrupted
#     except (pickle.UnpicklingError, EOFError, IOError):
#         return True  # File is corrupted

# for img in os.listdir("/content/imgcollect"):
#     output_file_path = '/content/rsamples/' + img[:-4] + '.pkl'

#     # Check if the output file already exists
#     if Path(output_file_path).exists() and not is_file_corrupted(output_file_path):
#         print(f"Skipping {img} as the output file already exists and is not corrupted.")
#         continue

#     print("starting new image")
#     imgfile = LandsatCustom("/content/imgcollect/" + img)
#     with open(output_file_path, 'wb') as file:
#         pickle.dump(imgfile.dataset, file)

#     gc.collect()

In [ ]:
# d = {}

# for img in os.listdir("/content/imgcollect"):
#   name = img[:-4]
#   d["{0}".format(name)] = LandsatCustom("/content/imgcollect/" + img)

In [ ]:
# train_dataset = CombinedDataset(d,2500)
# val_dataset = CombinedDataset(d,500)
# test_dataset = CombinedDataset(d,500)

In [ ]:
# train_loader = DataLoader(train_dataset, batch_size=64)
# val_loader = DataLoader(val_dataset, batch_size=64)
# test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
weights = ResNet18_Weights.LANDSAT_OLI_SR_MOCO
in_chans = weights.meta["in_chans"]
model = timm.create_model("resnet18", in_chans=in_chans, num_classes=12)
model.load_state_dict(weights.get_state_dict(progress=True), strict=False)
model = model.to(device)

for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
epoch = 0
val_accuracy = 0.0

while val_accuracy < 96:
      epoch += 1
      model.train()
      running_loss = 0.0

      for inputs, labels in train_loader:
          inputs, labels = inputs.to(device), labels.to(device)
          optimizer.zero_grad()

          outputs = model(inputs).to(device)
          labels -= 1
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      # Validate
      val_loss = 0.0
      correct = 0
      total = 0
      model.eval()

      for inputs, labels in val_loader:
          inputs, labels = inputs.to(device), labels.to(device)
          labels -= 1
          outputs = model(inputs).to(device)
          loss = criterion(outputs, labels)
          val_loss += loss.item()
          _,predicted = torch.max(outputs,1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      #Calculate Performance
      val_loss /= len(val_loader)
      val_accuracy = 100 * (correct / total)

      print(f'Epoch {epoch}, Train Loss: {running_loss / len(train_loader)}, Val Accuracy: {val_accuracy:.2f}%')

best_weights = model.state_dict()
torch.save(best_weights, 'best_model.pth')

In [ ]:
for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
epoch = 0
val_accuracy = 0.0

while val_accuracy < 98:
      epoch += 1
      model.train()
      running_loss = 0.0

      for inputs, labels in train_loader:
          inputs, labels = inputs.to(device), labels.to(device)
          optimizer.zero_grad()

          outputs = model(inputs).to(device)
          labels -= 1
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      # Validate
      val_loss = 0.0
      correct = 0
      total = 0
      model.eval()

      for inputs, labels in val_loader:
          inputs, labels = inputs.to(device), labels.to(device)
          labels -= 1
          outputs = model(inputs).to(device)
          loss = criterion(outputs, labels)
          val_loss += loss.item()
          _,predicted = torch.max(outputs,1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      #Calculate Performance
      val_loss /= len(val_loader)
      val_accuracy = 100 * (correct / total)

      print(f'Epoch {epoch}, Train Loss: {running_loss / len(train_loader)}, Val Accuracy: {val_accuracy:.2f}%')

best_weights = model.state_dict()
torch.save(best_weights, 'best_model.pth')

Epoch 1, Train Loss: 0.02669930965550671, Val Accuracy: 95.76%
Epoch 2, Train Loss: 0.018040026472822115, Val Accuracy: 97.04%
Epoch 3, Train Loss: 0.012648368942414879, Val Accuracy: 98.08%


In [ ]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
  for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs).to(device)
    _, predicted = torch.max(outputs,1)
    total += labels.size(0)
    labels -= 1
    correct += (predicted == labels).sum().item()

accuracy = 100 * (correct / total)
print(f'Test Accuracy: {accuracy:.2f}%')

In [ ]:
# with open('/content/rsamples/LC08_015029_20190201.pkl', 'rb') as file:
#     dataset = pickle.load(file)

In [ ]:
# class LandsatCustom(RasterDataset):
#     is_image = True
#     separate_files = False
#     all_bands = ["B1","B2","B3","B4","B5","B6","B7","B8","B9","B10","B11"]
#     rgb_bands = ["B4", "B3", "B2"]
#     filename_regex = r'(?P<satellite>[A-Za-z0-9]+)_(?P<number>\d+)_(?P<date>(?P<year>\d{4})(?P<month>\d{2})(?P<day>\d{2}))\.tif'

#     def __getitem__(self, index):
#         raster_data = super(LandsatCustom, self).__getitem__(index)
#         image = raster_data['image'][:7,:,:]
#         label = datetime.utcfromtimestamp(self.index.bounds[4]).month
#         return (image, label)

#     def get_samples(self):
#         xmin = self.index.bounds[0] + (self.index.bounds[1] - self.index.bounds[0])/5
#         xmax = self.index.bounds[1] - (self.index.bounds[1] - self.index.bounds[0])/5
#         ymin = self.index.bounds[2] + (self.index.bounds[3] - self.index.bounds[2])/5
#         ymax = self.index.bounds[3] - (self.index.bounds[3] - self.index.bounds[2])/5

#         newroi = torchgeo.datasets.BoundingBox(xmin, xmax, ymin, ymax, self.index.bounds[4], self.index.bounds[5])

#         sampler = RandomGeoSampler(self, roi=newroi, size=256, length=1)
#         dataset = []
#         for index in sampler:
#           sample = self[index]
#           dataset.append(sample)
#         return dataset[0]

In [ ]:
# !gsutil cp -r /content/rsamples/* gs://dlcv_finalproj_data1/

In [ ]:
folder_path = '/content/random_samples'

# List all files in the specified folder
file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.pkl')]
combined_data = []

for file_path in file_paths:
    with open(file_path, 'rb') as file:
        data_chunk = pickle.load(file)
        combined_data.extend(data_chunk)
    file.close()

# Output file path
output_file_path = '/content/combined_data.pkl'

# Save the combined data into a new .pkl file
with open(output_file_path, 'wb') as output_file:
    pickle.dump(combined_data, output_file)

In [ ]:
!gsutil cp /content/combined_data.pkl gs://dlcv_finalproj_data1/